# 演習
RNNを使って、小説を執筆しましょう。
RNNのモデルを構築し、文章を自動生成します。


## テキストデータの読み込み
Google ドライブからテキストデータを読み込みます。  
このノートブックと同じフォルダに、青空文庫から取得した江戸川乱歩の「怪人二十面相」のテキストデータ"kaijin_nijumenso.txt"がありますので、パスを指定して読み込みます。

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')
nov_dir = 'Udemy_activity/ai_master_course/Section_8/'  # このフォルダへのパス
nov_path = '/content/drive/My Drive/' + nov_dir + 'kaijin_nijumenso.txt'

# ファイルを読み込む
with open(nov_path, 'r') as f:
  nov_text = f.read()
  print(nov_text[:1000])  # 最初の1000文字のみ表示

## 正規表現による前処理
正規表現を使って、ルビなどを除去します。

In [ ]:
import re  # 正規表現に必要なライブラリ

text = re.sub("《[^》]+》", "", nov_text) # ルビの削除
text = re.sub("［[^］]+］", "", text) # 読みの注意の削除
text = re.sub("[｜ 　]", "", text) # | と全角半角スペースの削除
print("文字数", len(text))  # len() で文字列の文字数も取得可能

## 各設定
RNNの各設定です。  
ここは自由に変更しても大丈夫です。  

In [ ]:
n_rnn = 10  # 時系列の数
batch_size = 128
epochs = 60
n_mid = 128  # 中間層のニューロン数

## 文字のベクトル化
各文字をone-hot表現で表し、時系列の入力データおよび正解データを作成します。  
今回はRNNの最後の時刻の出力のみ利用するので、最後の出力に対応する正解のみ必要になります。

In [ ]:
import numpy as np

# インデックスと文字で辞書を作成
chars = sorted(list(set(text)))  # setで文字の重複をなくし、各文字をリストに格納する
print("文字数（重複無し）", len(chars))
char_indices = {}  # 文字がキーでインデックスが値
for i, char in enumerate(chars):
    char_indices[char] = i
indices_char = {}  # インデックスがキーで文字が値
for i, char in enumerate(chars):
    indices_char[i] = char
 
# 時系列データと、それから予測すべき文字を取り出します
time_chars = []
next_chars = []
for i in range(0, len(text) - n_rnn):
    time_chars.append(text[i: i + n_rnn])
    next_chars.append(text[i + n_rnn])
 
# 入力と正解をone-hot表現で表します
x = np.zeros((len(time_chars), n_rnn, len(chars)), dtype=np.bool)
t = np.zeros((len(time_chars), len(chars)), dtype=np.bool)
for i, t_cs in enumerate(time_chars):
    t[i, char_indices[next_chars[i]]] = 1  # 正解をone-hot表現で表す
    for j, char in enumerate(t_cs):
        x[i, j, char_indices[char]] = 1  # 入力をone-hot表現で表す
        
print("xの形状", x.shape)
print("tの形状", t.shape)

## モデルの構築
以下のセルに文章を生成するRNNのモデルを構築しましょう。

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, GRU

# 以下にRNNのモデルを構築する
model = Sequential()


## 文書生成用の関数
各エポックの終了後、文章を生成するための関数を記述します。  
LambdaCallbackを使って、エポック終了時に実行される関数を設定します。

In [ ]:
from keras.callbacks import LambdaCallback
 
def on_epoch_end(epoch, logs):
    print("エポック: ", epoch)

    beta = 5  # 確率分布を調整する定数
    prev_text = text[0:n_rnn]  # 入力に使う文字
    created_text = prev_text  # 生成されるテキスト
    
    print("シード: ", created_text)

    for i in range(400):
        # 入力をone-hot表現に
        x_pred = np.zeros((1, n_rnn, len(chars)))
        for j, char in enumerate(prev_text):
            x_pred[0, j, char_indices[char]] = 1
        
        # 予測を行い、次の文字を得る
        y = model.predict(x_pred)
        p_power = y[0] ** beta  # 確率分布の調整
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power))        
        next_char = indices_char[next_index]

        created_text += next_char
        prev_text = prev_text[1:] + next_char

    print(created_text)
    print()

# エポック終了後に実行される関数を設定
epock_end_callback= LambdaCallback(on_epoch_end=on_epoch_end)

## 学習
構築したモデルを使って、学習を行います。  
fit( )メソッドをではコールバックの設定をし、エポック終了後に関数が呼ばれるようにします。  
学習には時間がかかりますので、編集→ノートブックの設定のハードウェアアクセラレーターでGPUを選択しましょう。

In [ ]:
model = model
history = model.fit(x, t,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[epock_end_callback])

## 学習の推移
誤差の推移を確認します。

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']

plt.plot(np.arange(len(loss)), loss)
plt.show()

# 解答例
以下は解答例です。  
どうしてもわからない時のみ、参考にしましょう。  
GRUを使った文章生成のモデルになります。

In [ ]:
model = Sequential()
model.add(GRU(n_mid, input_shape=(n_rnn, len(chars))))
model.add(Dense(len(chars), activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer="adam")
print(model.summary())